📝 **Author:** Amirhossein Heydari - 📧 **Email:** amirhosseinheydari78@gmail.com - 📍 **Linktree:** [linktr.ee/mr_pylin](https://linktr.ee/mr_pylin)

---

# Dependencies

In [21]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchinfo import summary
from torchmetrics.classification import MulticlassAccuracy
from torchvision.datasets import CIFAR10
from torchvision.models import ResNet50_Weights, resnet50
from torchvision.transforms import v2

In [22]:
# set a seed for deterministic results
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
# check if cuda is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# log
device

# Pre-Processing

## Load Dataset

In [24]:
# initial transforms
transforms = v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])

In [ ]:
trainset = CIFAR10(root="../datasets", train=True, transform=transforms, download=False)
testset = CIFAR10(root="../datasets", train=False, transform=transforms, download=False)

# log
print("trainset:")
print(f"    -> trainset.data.shape    : {trainset.data.shape}")
print(f"    -> trainset.data.dtype    : {trainset.data.dtype}")
print(f"    -> type(trainset.data)    : {type(trainset.data)}")
print(f"    -> type(trainset.targets) : {type(trainset.targets)}")
print("-" * 50)
print("testset:")
print(f"    -> testset.data.shape     : {testset.data.shape}")
print(f"    -> testset.data.dtype     : {testset.data.dtype}")
print(f"    -> type(testset.data)     : {type(testset.data)}")
print(f"    -> type(testset.targets)  : {type(testset.targets)}")
print("-" * 50)
print(f"classes : {trainset.classes}")
print(f"trainset distribution : {np.unique(trainset.targets, return_counts=True)[1]}")
print(f"testset  distribution : {np.unique(testset.targets, return_counts=True)[1]}")

In [ ]:
# plot
fig, axs = plt.subplots(nrows=4, ncols=8, figsize=(12, 6), layout="compressed")
for i in range(4):
    for j in range(8):
        axs[i, j].imshow(trainset.data[i * 8 + j], cmap="gray")
        axs[i, j].set_title(trainset.classes[trainset.targets[i * 8 + j]])
        axs[i, j].axis("off")
plt.show()

## Normalization

In [ ]:
data = next(iter(DataLoader(trainset, batch_size=len(trainset))))[0]

train_mean = data.mean(axis=(0, 2, 3))
train_std = data.std(axis=(0, 2, 3))

del data

# log
print(f"train mean per channel : {train_mean}")
print(f"train std  per channel : {train_std}")

In [ ]:
transforms.transforms.append(v2.Normalize(mean=train_mean, std=train_std))

# log
print(f"trainset.transforms:\n{trainset.transforms}\n")
print(f"testset.transforms:\n{testset.transforms}")

In [ ]:
# log
print("before applying transform:")
print(f"    -> type(testset.data[0]) : {type(testset.data[0])}")
print(f"    -> testset.data[0].dtype : {testset.data[0].dtype}")
print(f"    -> testset.data[0].shape : {testset.data[0].shape}")
print("-" * 50)
print("after applying transform:")
print(f"    -> type(testset[0][0])   : {type(testset[0][0])}")
print(f"    -> testset[0][0].dtype   : {testset[0][0].dtype}")
print(f"    -> testset[0][0].shape   : {testset[0][0].shape}")

## DataLoader

In [30]:
batch_size = 64
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# Transfer Learning
   - resnet50 pretrained on IMAGENET1K_V1

In [ ]:
resnet = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

# log
resnet

In [ ]:
summary(resnet, input_size=(1, *trainset[0][0].shape), device="cpu")

## Extract a subset of Pretrained Model

In [ ]:
feature_extractor = nn.Sequential(*list(resnet.children())[:-1])

# log
for name, param in feature_extractor.named_parameters():
    print(f"{name:<23} - requires_grad: {param.requires_grad}")

## Freeze all transferred layers

In [ ]:
for name, param in resnet.named_parameters():
    param.requires_grad = False

# log
for name, param in resnet.named_parameters():
    print(f"{name:<28} - requires_grad : {param.requires_grad}")

## Transfer the model

In [ ]:
class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.feature_extractor = feature_extractor
        self.flatten = nn.Flatten(start_dim=1)
        self.classifier = nn.Sequential(nn.Linear(2048, 256), nn.ReLU(), nn.Linear(256, 10))

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.flatten(x)
        x = self.classifier(x)
        return x


# initialize the model
model = MyModel().to(device)

# log
model

In [ ]:
for name, param in model.named_parameters():
    print(f"{name:<41} - requires_grad : {param.requires_grad}")

In [ ]:
summary(model, input_size=(1, *trainset[0][0].shape))

# Set up remaining Hyperparameters

In [38]:
lr = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = Adam(params=model.parameters(), lr=lr)
num_epochs = 3

# Train Loop

In [39]:
train_acc_per_epoch = []
train_loss_per_epoch = []
train_acc = MulticlassAccuracy(num_classes=len(testset.classes), top_k=1).to(device)

In [ ]:
for epoch in range(num_epochs):

    model.train()
    train_loss = 0

    for x, y in trainloader:

        # send data to GPU
        x, y_true = x.to(device), y.to(device)

        # forward
        y_pred = model(x)
        loss = criterion(y_pred, y_true)

        # backward
        loss.backward()

        # update parameters
        optimizer.step()
        optimizer.zero_grad()

        # store loss and accuracy per iteration
        train_loss += loss.item() * len(x)
        train_acc.update(y_pred, y_true)

    # store loss and accuracy per epoch
    train_loss_per_epoch.append(train_loss / len(trainset))
    train_acc_per_epoch.append(train_acc.compute().item())
    train_acc.reset()

    # log
    print(
        f"epoch {epoch+1:0{len(str(num_epochs))}}/{num_epochs}  ->  train[loss: {train_loss_per_epoch[epoch]:.5f} - acc: {train_acc_per_epoch[epoch]:.2f}]"
    )